In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
from itertools import product
import colorsys
import networkx as nx
from functools import lru_cache
from tqdm import tqdm
import numpy as np
from matplotlib.collections import LineCollection

In [ ]:
ROAD_DENSITY = 0.85

num_zones = 16
num_trips = 20

width = int(200 * (num_zones**0.5))
height = int(200 * (num_zones**0.5))
num_horiz_roads = 4 * num_zones
num_vert_roads = 4 * num_zones

# Compute these
horiz_spacing = int(width / (num_horiz_roads - 1))
vert_spacing = int(height / (num_vert_roads - 1))

# Find the best approximate values for these variables that mean the roads are evenly spaced and fill the area
variables = list(
    product(
        range(width-5, width+6),
        range(height-5, height+6),
        range(num_horiz_roads-5, num_horiz_roads+6),
        range(num_vert_roads-5, num_vert_roads+6),
        range(horiz_spacing-5, horiz_spacing+6),
        range(vert_spacing-5, vert_spacing+6)
    )
)

def error_function(variables):
    width, height, num_horiz_roads, num_vert_roads, horiz_spacing, vert_spacing = variables
    horiz_error = abs(width - (num_horiz_roads - 1) * horiz_spacing) / width
    vert_error = abs(height - (num_vert_roads - 1) * vert_spacing) / height
    
    # Disqualify if total width or height taken up by roads spills over
    if (num_horiz_roads - 1) * horiz_spacing > width or (num_vert_roads - 1) * vert_spacing > height:
        return float('inf')
    
    return horiz_error + vert_error

best_variables = min(variables, key=error_function)
width, height, num_horiz_roads, num_vert_roads, horiz_spacing, vert_spacing = best_variables
print(width, height, num_horiz_roads, num_vert_roads, horiz_spacing, vert_spacing)

time_per_unit = 10
# traffic_function = lambda: 0.9 + random.random() * 0.2
traffic_function = lambda: 1

In [ ]:
# Generate random colours
colours = []
transparency = 0.5
for i in range(num_zones):
    hue = i / num_zones
    lightness = 0.75
    saturation = 0.9
    rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
    hex_colour = "#{:02x}{:02x}{:02x}{:02x}".format(
        int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255), int(transparency * 255)
    )
    colours.append(hex_colour)
random.shuffle(colours)

# Generate random names
prefixes = [
    "North",
    "South",
    "East",
    "West",
    "Central",
    "Lower",
    "Upper",
    "Old",
    "New",
    "Inner",
    "Outer",
    "Upper",
    "Lower",
    "Little",
    "Big",
    "Saint",
    "King",
    "Queen",
    "Prince",
    "Princess",
    "Royal",
    "Grand",
]
suffixes = [
    "District",
    "Quarter",
    "Heights",
    "Village",
    "Park",
    "Square",
    "Gardens",
    "Town",
    "City",
    "Vista",
    "Valley",
    "Hills",
    "Meadows",
    "Forest",
    "Grove",
    "Lake",
    "River",
    "Bay",
    "Harbor",
    "Port",
    "Beach",
    "Cove",
]

names = [f"{prefix} {suffix}" for prefix, suffix in product(prefixes, suffixes)]
random.shuffle(names)
names = names[: num_zones]

print("Example names:")
for name in names[:5]:
    print("- ", name)

In [ ]:
def draw_on_lines(graph, lines, colour):
    # Fetch all node positions in one go
    pos = {node: (data["x"], data["y"]) for node, data in graph.nodes(data=True)}
    
    # Convert lines into an array of coordinates
    segments = np.array([(pos[u], pos[v]) for u, v in lines])

    # Use LineCollection for faster bulk rendering
    lc = LineCollection(segments, colors=colour, linewidths=2, alpha=1)
    plt.gca().add_collection(lc)

def add_route(graph, route, colour, is_actual_route=False):
    pos = {node: (data["x"], data["y"]) for node, data in graph.nodes(data=True)}

    start, end = pos[route[0]], pos[route[-1]]

    if is_actual_route:
        plt.scatter(*start, color="green", s=100, zorder=5)
        plt.scatter(*end, color="red", s=100, zorder=5)
    else:
        plt.scatter(*start, color="blue", s=100, zorder=5)
        plt.scatter(*end, color="orange", s=100, zorder=5)

    draw_on_lines(graph, zip(route[:-1], route[1:]), colour)

def visualise(street_graph, zones, trips=[]):
    plt.figure(figsize=(15, 10))

    plt.xlim(0, width)
    plt.ylim(0, height)
    plt.gca().set_facecolor("#f8f9fa")

    draw_on_lines(street_graph, street_graph.edges(), "#8080807f")

    for zone in zones:
        rect = patches.Rectangle(
            (zone["x"], zone["y"]),
            zone["width"],
            zone["height"],
            linewidth=1,
            edgecolor="#333333",
            facecolor=zone["colour"],
            alpha=0.5,
        )
        plt.gca().add_patch(rect)
        plt.text(
            zone["x"] + zone["width"] / 2,
            zone["y"] + zone["height"] / 2,
            zone["name"],
            ha="center",
            va="center",
            fontweight="bold",
        )

    for trip in trips:
        add_route(street_graph, trip["route"], "black", is_actual_route=True)

        if trip["estimated_route"] is not None:
            add_route(street_graph, trip["estimated_route"], "black", is_actual_route=False)

    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# Generate random zones
min_size = min(width, height) // (num_zones // 2)

def split_area(x, y, width, height, remaining_zones):
    if remaining_zones <= 1 or min(width, height) < min_size:
        return [{"x": x, "y": y, "width": width, "height": height}]

    split_vertically = width > height
    split_range = (width if split_vertically else height) // 3
    split_pos = random.randint(split_range, 2 * split_range)

    first_half = remaining_zones // 2
    second_half = remaining_zones - first_half

    if split_vertically:
        return split_area(x, y, split_pos, height, first_half) + split_area(
            x + split_pos, y, width - split_pos, height, second_half
        )
    else:
        return split_area(x, y, width, split_pos, first_half) + split_area(
            x, y + split_pos, width, height - split_pos, second_half
        )

base_zones = split_area(0, 0, width, height, num_zones)
zones = [
    {
        "id": i,
        "x": z["x"],
        "y": z["y"],
        "width": z["width"],
        "height": z["height"],
        "name": name,
        "colour": colour,
    }
    for i, (z, name, colour) in enumerate(
        zip(base_zones, names, colours)
    )
]

visualise(nx.Graph(), zones)

In [ ]:
# Generate the street network
street_graph = nx.Graph()

# Create grid nodes
for i in range(num_horiz_roads + 1):
    for j in range(num_vert_roads + 1):
        x = i * horiz_spacing
        y = j * vert_spacing
        node_id = f"{x},{y}"
        street_graph.add_node(node_id, x=x, y=y)

# Connect horizontally
for i in range(num_horiz_roads):
    for j in range(num_vert_roads + 1):
        x1 = i * horiz_spacing
        x2 = (i + 1) * horiz_spacing
        y = j * vert_spacing
        node1 = f"{x1},{y}"
        node2 = f"{x2},{y}"
        distance = horiz_spacing
        street_graph.add_edge(node1, node2, weight=distance)

# Connect vertically
for i in range(num_horiz_roads + 1):
    for j in range(num_vert_roads):
        x = i * horiz_spacing
        y1 = j * vert_spacing
        y2 = (j + 1) * vert_spacing
        node1 = f"{x},{y1}"
        node2 = f"{x},{y2}"
        distance = vert_spacing
        street_graph.add_edge(node1, node2, weight=distance)

# Remove random edges
edges = list(street_graph.edges())
num_to_remove = int(len(edges) * (1-ROAD_DENSITY))
edges_to_remove = random.sample(edges, num_to_remove)
street_graph.remove_edges_from(edges_to_remove)

visualise(street_graph, zones)

In [ ]:
def is_point_in_zone(point, zone):
    x, y = point
    return (
        x >= zone["x"]
        and x <= zone["x"] + zone["width"]
        and y >= zone["y"]
        and y <= zone["y"] + zone["height"]
    )

In [ ]:
# Precompute street points inside each zone
@lru_cache(maxsize=None)
def get_street_points_in_zone(zone_id):
    zone = next((z for z in zones if z["id"] == zone_id), None)
    if not zone:
        return []

    points_in_zone = []
    for node_id, data in street_graph.nodes(data=True):
        if is_point_in_zone((data["x"], data["y"]), zone):
            points_in_zone.append({"x": data["x"], "y": data["y"], "id": node_id})

    return points_in_zone

points_in_zone = {
    zone["id"]: get_street_points_in_zone(zone["id"])
    for zone in zones
}


In [ ]:
# Generate trips
trips = []

for i in range(num_trips):
    start_zone_index = random.randint(0, len(zones) - 1)
    start_zone = zones[start_zone_index]

    end_zone_index = random.randint(0, len(zones) - 1)
    end_zone = zones[end_zone_index]

    start_point = random.choice(points_in_zone[start_zone["id"]])
    end_point = random.choice(points_in_zone[end_zone["id"]])
    try:
        route = nx.shortest_path(street_graph, start_point["id"], end_point["id"], weight="weight")
    except nx.NetworkXNoPath:
        continue
    distance = nx.shortest_path_length(street_graph, start_point["id"], end_point["id"], weight="weight")
    base_time = distance / time_per_unit
    travel_time = traffic_function() * base_time

    trips.append(
        {
            "id": i,
            "start_zone_id": start_zone["id"],
            "start_zone_name": start_zone["name"],
            "end_zone_id": end_zone["id"],
            "end_zone_name": end_zone["name"],
            "travel_time": travel_time,
            "distance": distance,
            "real_start_point": dict(start_point),
            "real_end_point": dict(end_point),
            "estimated_start_point": None,
            "estimated_end_point": None,
            "route": route,
            "estimated_route": None,
        }
    )

print("Example trips:")
for trip in trips[:5]:
    print(
        f"- {trip['start_zone_name']} to {trip['end_zone_name']} in {trip['travel_time']:.2f} units"
    )

visualise(street_graph, zones, trips[:5])

In [ ]:
# Compute all possible route lengths to and from all points in the street graph
all_routes = {}

for source in tqdm(street_graph.nodes(), desc="Computing Distance-Only Dijkstra"):
    all_routes[source] = nx.single_source_dijkstra_path_length(street_graph, source, weight="weight")


In [ ]:
# Get the area that a given trip could start and end in
#     In essense, we're getting all points in the start zone and end zone
#     that have a valid path to the start and end points of the
#     trip respectively

def get_valid_area(trip):
    start_points = get_street_points_in_zone(trip["start_zone_id"])
    end_points = get_street_points_in_zone(trip["end_zone_id"])

    # A start point is valid if a path exists starting from it and ends in the end zone, that is the correct length
    valid_start_points = [
        point
        for point in start_points
        if any(
            end_point["id"] in all_routes[point["id"]] and abs(all_routes[point["id"]][end_point["id"]] - trip["distance"]) < 1e-6
            for end_point in end_points
        )
    ]

    valid_end_points = [
        point
        for point in end_points
        if any(
            start_point["id"] in all_routes[point["id"]] and abs(all_routes[point["id"]][start_point["id"]] - trip["distance"]) < 1e-6
            for start_point in start_points
        )
    ]

    fraction_start = len(valid_start_points) / len(start_points)
    fraction_end = len(valid_end_points) / len(end_points)
    return valid_start_points, valid_end_points, fraction_start, fraction_end

# Example
trip = trips[6]
valid_start_points, valid_end_points, fraction_start, fraction_end = get_valid_area(trip)

# Get average thinning factors
fraction_start_sum = sum(get_valid_area(trip)[2] for trip in trips)
fraction_end_sum = sum(get_valid_area(trip)[3] for trip in trips)

print(f"Trip {trip['id']} can start from {len(valid_start_points)} points and end at {len(valid_end_points)} points")
print(f"Thinning factor: {fraction_start:.2f} for start and {fraction_end:.2f} for end")

print(f"Average thinning factor: {fraction_start_sum / len(trips):.2f} for start and {fraction_end_sum / len(trips):.2f} for end")

In [ ]:
# Predict the start and end points of each trip in each zone

# This works by randomly selecting a start and end point in each zone
# Then, iteratively move the points around the zone wrt. the graph
# So the trip length matches the inputed travel time

def crude_predict_start_end_points(trip):
    start_zone = next((z for z in zones if z["id"] == trip["start_zone_id"]), None)
    end_zone = next((z for z in zones if z["id"] == trip["end_zone_id"]), None)

    start_point = random.choice(points_in_zone[start_zone["id"]])
    end_point = random.choice(points_in_zone[end_zone["id"]])

    route = nx.shortest_path(street_graph, start_point["id"], end_point["id"], weight="weight")
    distance = all_routes[start_point["id"]][end_point["id"]]
    distance = nx.shortest_path_length(street_graph, start_point["id"], end_point["id"], weight="weight")
    base_time = distance / time_per_unit
    travel_time = traffic_function() * base_time
    distance_error = abs(distance - trip["travel_time"])
    time_error = abs(travel_time - trip["travel_time"])

    for _ in range(500):
        new_start_point = random.choice(points_in_zone[start_zone["id"]])
        new_end_point = random.choice(points_in_zone[end_zone["id"]])

        try:
            new_route = nx.shortest_path(street_graph, new_start_point["id"], new_end_point["id"], weight="weight")
        except nx.NetworkXNoPath:
            continue
        new_distance = nx.shortest_path_length(street_graph, new_start_point["id"], new_end_point["id"], weight="weight")
        new_base_time = new_distance / time_per_unit
        new_travel_time = traffic_function() * new_base_time
        
        # Is the new route's travel time and distance closer to the target?
        new_distance_error = abs(new_distance - trip["distance"])
        new_time_error = abs(new_travel_time - trip["travel_time"])

        has_distance_improved = new_distance_error < distance_error and new_time_error <= time_error
        has_time_improved = new_time_error < time_error and new_distance_error <= distance_error
        
        if has_distance_improved or has_time_improved:
            print("Found a better route with distance error", new_distance_error, "and time error", new_time_error)
            route = new_route
            distance = new_distance
            base_time = new_base_time
            travel_time = new_travel_time
            start_point = new_start_point
            end_point = new_end_point
            distance_error = new_distance_error
            time_error = new_time_error

    # Assign our inferred start and end points to the trip
    trip["estimated_start_point"] = start_point
    trip["estimated_end_point"] = end_point
    trip["estimated_route"] = route

# Predict the start and end points for the first trip, displaying the guessed and actual trip routes
trip = trips[1]
crude_predict_start_end_points(trip)

estimated_route = trip["estimated_route"]
actual_route = trip["route"]

estimated_route_length = nx.shortest_path_length(street_graph, estimated_route[0], estimated_route[-1], weight="weight")
actual_route_length = nx.shortest_path_length(street_graph, actual_route[0], actual_route[-1], weight="weight")

print("Estimated route length:", estimated_route_length)
print("Actual route length:", actual_route_length)

print("Generating graph...")
visualise(street_graph, zones, [trip])